In [ ]:
{'data': [
    {'nickname': '关伯兰', 'remark': '关伯兰', 'user_id': 474527445}, 
    {'nickname': '🍁🍁🍁', 'remark': '🍁🍁🍁', 'user_id': 577003680}, 
], 'message': '', 'retcode': 0, 'status': 'ok'}

1.用户消息至控制器DispatcherServlet
2.DispatcherServlet进行加工处理后调用处理器映射器HandlerMapping。
3.HandlerMapping根据消息找到具体的处理器，生成响应消息返回给DispatcherServlet。
4.DispatcherServlet根据处理器Handler获取处理器适配器HandlerAdapter执行HandlerAdapter处理一系列的操作，如：参数封装，数据格式转换，数据验证等操作
5.执行处理器Handler(Controller，也叫页面控制器)。
6.Handler执行完成返回ModelAndView
7.HandlerAdapter将Handler执行结果ModelAndView返回到DispatcherServlet
8.DispatcherServlet将ModelAndView传给ViewReslover视图解析器
9.ViewReslover解析后返回具体View
10.DispatcherServlet对View进行渲染视图（即将模型数据model填充至视图中）。
11.DispatcherServlet响应用户。


In [ ]:
{
    'post_type': 'message', 
    'message_type': 'private', 
    'time': 1694183059, 
    'self_id': 2762018040, 
    'sub_type': 'friend', 
    'target_id': 2762018040, 
    'message': '你好', 
    'raw_message': '你好', 
    'font': 0, 
    'sender': {'age': 0, 'nickname': 'Aki-Polaris', 'sex': 'unknown', 'user_id': 815049548}, 
    'message_id': -2001115448, 
    'user_id': 815049548
}
message_info = {
    'post_type': 'message', 
    'message_type': 'group', 
    'time': 1694395091, 
    'self_id': 2762018040, 
    'sub_type': 'normal', 
    'anonymous': None, 
    'message': '[CQ:image,file=138fd15bdabbcfb4c3daa148555fe447.image,subType=1,url=https://gchat.qpic.cn/gchatpic_new/1352402688/830954892-2582910460-138FD15BDABBCFB4C3DAA148555FE447/0?term=255&amp;is_origin=0]', 
    'message_seq': 81567, 
    'raw_message': '[CQ:image,file=138fd15bdabbcfb4c3daa148555fe447.image,subType=1,url=https://gchat.qpic.cn/gchatpic_new/1352402688/830954892-2582910460-138FD15BDABBCFB4C3DAA148555FE447/0?term=255&amp;is_origin=0]', 
    'font': 0, 
    'group_id': 830954892, 
    'sender': {'age': 0, 'area': '', 'card': '', 'level': '', 'nickname': '离岱和葛饰应为', 'role': 'member', 'sex': 'unknown', 'title': '', 'user_id': 1352402688}, 
    'user_id': 1352402688, 
    'message_id': -185279243
}

In [ ]:
from ltp import LTP
ltp = LTP('LTP/small')
message = '明天天怎么样呀'

In [ ]:
from service.MatchSys.object_definition import Statement
from service.MatchSys.utils import import_module


class MessageAdapter(object):
    """
    This is an abstract class that represents the interface
    that all message adapters should implement.
    """
    def __init__(self, **kwargs) -> None:
        from service.MatchSys.object_definition import Statement
        from ltp import LTP

        # 初始化预处理程序
        preprocessors = kwargs.get('preprocessors', ['jionlp.clean_text'])
        self.preprocessors = []
        for preprocessor in preprocessors:
            self.preprocessors.append(import_module(preprocessor))
        
        model_path = kwargs.get('model_path', 'LTP/small')
        self.ltp = LTP(model_path)
    class AdapterMethodNotImplementedError(NotImplementedError):
        """
        An exception to be raised when an adapter method has not been implemented.
        Typically this indicates that the developer is expected to implement the
        method in a subclass.
        """

        def __init__(self, message='This method must be overridden in a subclass method.'):
            """
            Set the message for the exception.
            """
            super().__init__(message)

    def check(self, message):
        if message is None or message == '':
            raise self.ChatBotException(
                'Either a statement object or a "text" keyword '
                'argument is required. Neither was provided.'
            )
        
        return True

    def process(self, message):
        # read the message

        # Get Text message
        input_statement = self.text_process(text=message)

        # Add Other Info

        # 获取Statement
        raise self.AdapterMethodNotImplementedError()

    def text_process(self, text,**kwargs):
        """Return Search Text

        Args:
            text (_type_): _description_

        Returns:
            _type_: Statement
        """
        # 清理文本
        for preprocessor in self.preprocessors:
            text = preprocessor(text)
        kwargs['text'] = text
        # 分词
        result = self.ltp.pipeline('你觉得A怎么样', tasks = ["cws","srl"])
        kwargs['search_text'] = result.cws
        t = result.srl[0]
        for item in result.srl:
            if len(t['arguments']) > len(item['arguments']):
                t = item
        kwargs['intent'] = t

        input_statement = Statement(**kwargs)
        return input_statement

    @property
    def class_name(self):
        """
        Return the name of the current logic adapter class.
        This is typically used for logging and debugging.
        """
        return str(self.__class__.__name__)

In [ ]:
class TextMessageAdapter(MessageAdapter):
    """
    This is an abstract class that represents the interface
    that all message adapters should implement.
    """
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

    def process(self, message):
        # read the message

        # Get Text message
        input_statement = self.text_process(text=message)

        # Add Other Info

        # 获取Statement
        return input_statement

In [ ]:
message_adapter = TextMessageAdapter(model_path=r'D:\Code\MyLongTimeProject\A\model')

In [ ]:
from service.MatchSys.trainers import Trainer
from service.MatchSys.utils import print_progress_bar

class QATrainer(Trainer):
    """
    Allows a chat bot to be trained using a list of strings
    where the list represents a conversation.
    """

    def train(self, conversation, **kwargs):
        """
        {Q:[A1,A2...]}
        Train the chat bot based on the provided list of
        statements that represents a single conversation.
        """
        source = kwargs.get('source', 'TRAIN_DATA')
        conversation_text = kwargs.get('conversation', 'TRAIN_DATA')
        statements_to_create = []
        for index,Q in enumerate(conversation):
            if self.show_training_progress:
                print_progress_bar(
                    'QA Trainer',
                    index + 1, len(conversation)
                )
    
            statement = self.chatbot.message_adapter.process(Q)
            statement.next_id=-statement.id
            statement.conversation=conversation_text
            statement.type_of='Q'
            statement.source=source
            statement.persona='user'
            statements_to_create.append(statement)
            for A in conversation[Q]:
                statement = self.chatbot.message_adapter.process(A)
                statement.previous_id=-statement.id
                statement.conversation=conversation_text
                statement.type_of='A'
                statement.source=source
                statement.persona='bot:'+self.chatbot.name
                statements_to_create.append(statement)
        self.chatbot.storage.create_many(statements_to_create)
        self.chatbot.docvector_tool.train(statements_to_create)

In [ ]:
from service.MatchSys.match_sys import MatchSys


matchsys = MatchSys(name='Alice',model_path=r'D:\Code\MyLongTimeProject\A\model')

In [ ]:
qatrain = QATrainer(matchsys)

In [30]:
import pandas as pd
a = pd.read_csv(r'data1.txt',
                sep="$",
                engine='python'
                )
a.head(5)
a.to_csv(r'data1.csv',)

In [1]:
a = {'a':[1,2,3], 'b':[4,5,6], 'c':[7,8]}
print(a.keys())
print(a.values())

dict_keys(['a', 'b', 'c'])
dict_values([[1, 2, 3], [4, 5, 6], [7, 8]])


In [3]:
temp = {}
values = []
count = 0
for key,value in a.items():
    count = len(value) + count
    temp[key] = count
    for i in value:
        values.append(i)

In [19]:
print(values)
values[0:7]

[1, 2, 3, 4, 5, 6, 7, 8]


[1, 2, 3, 4, 5, 6, 7]